In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, StratifiedKFold
import shutil

from torch.utils.data import DataLoader, Dataset
import datasets, transformers
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification, AutoTokenizer

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [6]:
test_df = pd.read_csv(f"/kaggle/input/us-patent-phrase-to-phrase-matching/test.csv")
titles = pd.read_csv('/kaggle/input/cpc-codes/titles.csv')
test_df = test_df.merge(titles, left_on='context', right_on='code')
test_df['input'] = test_df['title']+' '+test_df['anchor']
test_df.head()

,id,anchor,target,context,code,title,section,class,subclass,group,main_group,input
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02,G02,OPTICS,G,2.0,NaN,NaN,NaN,OPTICS opc drum
1,5203a36c501f1b7c,generate in layer,generate by layer,G02,G02,OPTICS,G,2.0,NaN,NaN,NaN,OPTICS generate in layer
2,7aa5908a77a7ec24,el display,illumination,G02,G02,OPTICS,G,2.0,NaN,NaN,NaN,OPTICS el display
3,09e418c93a776564,adjust gas flow,altering gas flow,F23,F23,COMBUSTION APPARATUS; COMBUSTION PROCESSES,F,23.0,NaN,NaN,NaN,COMBUSTION APPARATUS; COMBUSTION PROCESSES adj...
4,36baf228038e314b,lower trunnion,lower locating,B60,B60,VEHICLES IN GENERAL,B,60.0,NaN,NaN,NaN,VEHICLES IN GENERAL lower trunnion


In [8]:
tokenizer = AutoTokenizer.from_pretrained(f'/kaggle/input/uspppm-bert-train/uspppm_0')

In [10]:
class InferDataset(Dataset):
    def __init__(self, df):
        self.inputs = df['input'].values.astype(str)
        self.targets = df['target'].values.astype(str)

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, item):
        inputs = self.inputs[item]
        targets = self.targets[item]
        
        return {
        **tokenizer( inputs, targets )
    }

In [11]:
predictions = []

for fold in range(4):
    te_dataset = InferDataset(test_df)
    model = AutoModelForSequenceClassification.from_pretrained(f'/kaggle/input/uspppm-bert-train/uspppm_{fold}', num_labels=1)
    trainer = Trainer(
            model,
            tokenizer=tokenizer
        )

    outputs = trainer.predict(te_dataset)
    prediction = outputs.predictions.reshape(-1)
    predictions.append(prediction)
    
predictions = np.mean(predictions, axis=0)
submission = datasets.Dataset.from_dict({
    'id': test_df['id'],
    'score': predictions,
})

submission.to_csv('submission.csv', index=False)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

1017